In [1]:
from PIL import Image, ImageEnhance, ImageFilter  
import os
from glob import glob

import pandas as pd
import numpy as np
import random

In [2]:
def create_folder(path, folder = ""):
    
    final_path = os.path.join(path, folder)
    
    if not os.path.isdir(final_path):
        os.makedirs (final_path)
    
    return final_path

In [3]:
def images_count(path_folder):
    lst = os.listdir(path_folder) 
    count = len(lst)
    return count

In [4]:
# https://www.codementor.io/@isaib.cicourel/image-manipulation-in-python-du1089j1u

# Return color value depending on quadrant and saturation
def get_saturation(value, quadrant):
  if value > 223:
    return 255
  elif value > 159:
    if quadrant != 1:
      return 255

    return 0
  elif value > 95:
    if quadrant == 0 or quadrant == 3:
      return 255

    return 0
  elif value > 32:
    if quadrant == 1:
      return 255

    return 0
  else:
    return 0

# Get the pixel from the given image
def get_pixel(image, i, j):
  # Inside image bounds?
  width, height = image.size
  if i > width or j > height:
    return None

  # Get Pixel
  pixel = image.getpixel((i, j))
  return pixel

# Create a dithered version of the image
def convert_dithering(image):
    # Get size
    width, height = image.size

    # Create new Image and a Pixel Map
    new = create_image_white(width, height)
    pixels = new.load()

    # Transform to half tones
    for i in range(0, width, 2):
        for j in range(0, height, 2):
          # Get Pixels
          p1 = get_pixel(image, i, j)
          p2 = get_pixel(image, i, j + 1)
          p3 = get_pixel(image, i + 1, j)
          p4 = get_pixel(image, i + 1, j + 1)

          # Color Saturation by RGB channel
          red   = (p1[0] + p2[0] + p3[0] + p4[0]) / 4
          green = (p1[1] + p2[1] + p3[1] + p4[1]) / 4
          blue  = (p1[2] + p2[2] + p3[2] + p4[2]) / 4

          # Results by channel
          r = [0, 0, 0, 0]
          g = [0, 0, 0, 0]
          b = [0, 0, 0, 0]

          # Get Quadrant Color
          for x in range(0, 4):
            r[x] = get_saturation(red, x)
            g[x] = get_saturation(green, x)
            b[x] = get_saturation(blue, x)

          # Set Dithered Colors
          pixels[i, j]         = (r[0], g[0], b[0])
          pixels[i, j + 1]     = (r[1], g[1], b[1])
          pixels[i + 1, j]     = (r[2], g[2], b[2])
          pixels[i + 1, j + 1] = (r[3], g[3], b[3])

    # Return new image
    return new

In [5]:
def convert_brightness(image, brightness):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness)
    

In [6]:
def image_name_brightness(name, brightness):
    return ''.join([name, "_", f"{brightness}".replace('.', '_')])

In [7]:
def image_name_dithering(name):
    return ''.join([name, "_dither"])

In [8]:
def image_name_low_quality(name):
    return ''.join([name, "_low"])

In [9]:
def save_image(img, path, name, fillcolor,  quality=75, extension='png'):

    
    img = img.rotate(random.randint(5, 175), Image.NEAREST, expand = False, fillcolor = fillcolor)
    img.save(f"{path}/{name}.{extension}", quality=quality)

In [10]:
# Create a new image with the given size
def create_image_white(i, j):
  image = Image.new("RGB", (i, j), "white")
  return image

In [11]:
def create_image(path, name, r, g, b, override = False, width = 128, height = 128, brightness = 1, extension='png'):
    print(path, name, r, g, b, brightness)
    create_folder(path)
    
    name_brightness = image_name_brightness(name, brightness)
    name_dithering = image_name_dithering(name_brightness)
    name_low_quality = image_name_low_quality(name_brightness)
    
    if glob(f"{path}/{image_name_brightness}.{extension}"):
        #print("already exists")
        if override:
            pass
            
    else:
        #print("let's go !")
        img  = Image.new( mode = "RGB", size = (width, height), color = (r, g, b) )
        
        img = convert_brightness(img, brightness)
        save_image(img, path, name_brightness, (r, g, b))
        
         
        save_image(img, path, name_low_quality, (r, g, b), 1)
        
        # Baisse les perfs
        img = convert_dithering(img)
        save_image(img, path, name_dithering, (r, g, b))
        
        
    

In [12]:

df = pd.read_csv("./dataset/html_color_names.csv", sep=";")

df.head()

,color,enable,color_name,hex_code,r,g,b,remark
0,red,1,IndianRed,#CD5C5C,205,92,92,NaN
1,red,1,LightCoral,#F08080,240,128,128,NaN
2,red,1,Salmon,#FA8072,250,128,114,NaN
3,red,1,DarkSalmon,#E9967A,233,150,122,NaN
4,red,1,LightSalmon,#FFA07A,255,160,122,NaN


In [13]:
images_folder = "./dataset/images/"

In [14]:

for index, row in df.iterrows():
    for i in range(5, 19):
        if row['enable'] == 1:
            create_image(
                f"{images_folder}{row['color']}", 
                row['color_name'], 
                r = row['r'], 
                g = row['g'], 
                b = row['b'], 
                brightness=i/10)
    

./dataset/images/red IndianRed 205 92 92 0.5
./dataset/images/red IndianRed 205 92 92 0.6
./dataset/images/red IndianRed 205 92 92 0.7
./dataset/images/red IndianRed 205 92 92 0.8
./dataset/images/red IndianRed 205 92 92 0.9
./dataset/images/red IndianRed 205 92 92 1.0
./dataset/images/red IndianRed 205 92 92 1.1
./dataset/images/red IndianRed 205 92 92 1.2
./dataset/images/red IndianRed 205 92 92 1.3
./dataset/images/red IndianRed 205 92 92 1.4
./dataset/images/red IndianRed 205 92 92 1.5
./dataset/images/red IndianRed 205 92 92 1.6
./dataset/images/red IndianRed 205 92 92 1.7
./dataset/images/red IndianRed 205 92 92 1.8
./dataset/images/red LightCoral 240 128 128 0.5
./dataset/images/red LightCoral 240 128 128 0.6
./dataset/images/red LightCoral 240 128 128 0.7
./dataset/images/red LightCoral 240 128 128 0.8
./dataset/images/red LightCoral 240 128 128 0.9
./dataset/images/red LightCoral 240 128 128 1.0
./dataset/images/red LightCoral 240 128 128 1.1
./dataset/images/red LightCoral 24

./dataset/images/pink MediumVioletRed 199 21 133 0.6
./dataset/images/pink MediumVioletRed 199 21 133 0.7
./dataset/images/pink MediumVioletRed 199 21 133 0.8
./dataset/images/pink MediumVioletRed 199 21 133 0.9
./dataset/images/pink MediumVioletRed 199 21 133 1.0
./dataset/images/pink MediumVioletRed 199 21 133 1.1
./dataset/images/pink MediumVioletRed 199 21 133 1.2
./dataset/images/pink MediumVioletRed 199 21 133 1.3
./dataset/images/pink MediumVioletRed 199 21 133 1.4
./dataset/images/pink MediumVioletRed 199 21 133 1.5
./dataset/images/pink MediumVioletRed 199 21 133 1.6
./dataset/images/pink MediumVioletRed 199 21 133 1.7
./dataset/images/pink MediumVioletRed 199 21 133 1.8
./dataset/images/pink PaleVioletRed 219 112 147 0.5
./dataset/images/pink PaleVioletRed 219 112 147 0.6
./dataset/images/pink PaleVioletRed 219 112 147 0.7
./dataset/images/pink PaleVioletRed 219 112 147 0.8
./dataset/images/pink PaleVioletRed 219 112 147 0.9
./dataset/images/pink PaleVioletRed 219 112 147 1.0

./dataset/images/yellow LightGoldenrodYellow 250 250 210 0.9
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.0
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.1
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.2
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.3
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.4
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.5
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.6
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.7
./dataset/images/yellow LightGoldenrodYellow 250 250 210 1.8
./dataset/images/yellow PapayaWhip 255 239 213 0.5
./dataset/images/yellow PapayaWhip 255 239 213 0.6
./dataset/images/yellow PapayaWhip 255 239 213 0.7
./dataset/images/yellow PapayaWhip 255 239 213 0.8
./dataset/images/yellow PapayaWhip 255 239 213 0.9
./dataset/images/yellow PapayaWhip 255 239 213 1.0
./dataset/images/yellow PapayaWhip 255 239 213 1.1
./dataset/images/yellow PapayaWhi

./dataset/images/purple Fuchsia 255 0 255 1.1
./dataset/images/purple Fuchsia 255 0 255 1.2
./dataset/images/purple Fuchsia 255 0 255 1.3
./dataset/images/purple Fuchsia 255 0 255 1.4
./dataset/images/purple Fuchsia 255 0 255 1.5
./dataset/images/purple Fuchsia 255 0 255 1.6
./dataset/images/purple Fuchsia 255 0 255 1.7
./dataset/images/purple Fuchsia 255 0 255 1.8
./dataset/images/purple Magenta 255 0 255 0.5
./dataset/images/purple Magenta 255 0 255 0.6
./dataset/images/purple Magenta 255 0 255 0.7
./dataset/images/purple Magenta 255 0 255 0.8
./dataset/images/purple Magenta 255 0 255 0.9
./dataset/images/purple Magenta 255 0 255 1.0
./dataset/images/purple Magenta 255 0 255 1.1
./dataset/images/purple Magenta 255 0 255 1.2
./dataset/images/purple Magenta 255 0 255 1.3
./dataset/images/purple Magenta 255 0 255 1.4
./dataset/images/purple Magenta 255 0 255 1.5
./dataset/images/purple Magenta 255 0 255 1.6
./dataset/images/purple Magenta 255 0 255 1.7
./dataset/images/purple Magenta 25

./dataset/images/purple DarkSlateBlue 72 61 139 1.4
./dataset/images/purple DarkSlateBlue 72 61 139 1.5
./dataset/images/purple DarkSlateBlue 72 61 139 1.6
./dataset/images/purple DarkSlateBlue 72 61 139 1.7
./dataset/images/purple DarkSlateBlue 72 61 139 1.8
./dataset/images/purple MediumSlateBlue 123 104 238 0.5
./dataset/images/purple MediumSlateBlue 123 104 238 0.6
./dataset/images/purple MediumSlateBlue 123 104 238 0.7
./dataset/images/purple MediumSlateBlue 123 104 238 0.8
./dataset/images/purple MediumSlateBlue 123 104 238 0.9
./dataset/images/purple MediumSlateBlue 123 104 238 1.0
./dataset/images/purple MediumSlateBlue 123 104 238 1.1
./dataset/images/purple MediumSlateBlue 123 104 238 1.2
./dataset/images/purple MediumSlateBlue 123 104 238 1.3
./dataset/images/purple MediumSlateBlue 123 104 238 1.4
./dataset/images/purple MediumSlateBlue 123 104 238 1.5
./dataset/images/purple MediumSlateBlue 123 104 238 1.6
./dataset/images/purple MediumSlateBlue 123 104 238 1.7
./dataset/im

...

./dataset/images/blue DarkTurquoise 0 206 209 1.2
./dataset/images/blue DarkTurquoise 0 206 209 1.3
./dataset/images/blue DarkTurquoise 0 206 209 1.4
./dataset/images/blue DarkTurquoise 0 206 209 1.5
./dataset/images/blue DarkTurquoise 0 206 209 1.6
./dataset/images/blue DarkTurquoise 0 206 209 1.7
./dataset/images/blue DarkTurquoise 0 206 209 1.8
./dataset/images/blue SteelBlue 70 130 180 0.5
./dataset/images/blue SteelBlue 70 130 180 0.6
./dataset/images/blue SteelBlue 70 130 180 0.7
./dataset/images/blue SteelBlue 70 130 180 0.8
./dataset/images/blue SteelBlue 70 130 180 0.9
./dataset/images/blue SteelBlue 70 130 180 1.0
./dataset/images/blue SteelBlue 70 130 180 1.1
./dataset/images/blue SteelBlue 70 130 180 1.2
./dataset/images/blue SteelBlue 70 130 180 1.3
./dataset/images/blue SteelBlue 70 130 180 1.4
./dataset/images/blue SteelBlue 70 130 180 1.5
./dataset/images/blue SteelBlue 70 130 180 1.6
./dataset/images/blue SteelBlue 70 130 180 1.7
./dataset/images/blue SteelBlue 70 130 

./dataset/images/blue Blue 0 0 255 1.1
./dataset/images/blue Blue 0 0 255 1.2
./dataset/images/blue Blue 0 0 255 1.3
./dataset/images/blue Blue 0 0 255 1.4
./dataset/images/blue Blue 0 0 255 1.5
./dataset/images/blue Blue 0 0 255 1.6
./dataset/images/blue Blue 0 0 255 1.7
./dataset/images/blue Blue 0 0 255 1.8
./dataset/images/blue MediumBlue 0 0 205 0.5
./dataset/images/blue MediumBlue 0 0 205 0.6
./dataset/images/blue MediumBlue 0 0 205 0.7
./dataset/images/blue MediumBlue 0 0 205 0.8
./dataset/images/blue MediumBlue 0 0 205 0.9
./dataset/images/blue MediumBlue 0 0 205 1.0
./dataset/images/blue MediumBlue 0 0 205 1.1
./dataset/images/blue MediumBlue 0 0 205 1.2
./dataset/images/blue MediumBlue 0 0 205 1.3
./dataset/images/blue MediumBlue 0 0 205 1.4
./dataset/images/blue MediumBlue 0 0 205 1.5
./dataset/images/blue MediumBlue 0 0 205 1.6
./dataset/images/blue MediumBlue 0 0 205 1.7
./dataset/images/blue MediumBlue 0 0 205 1.8
./dataset/images/blue DarkBlue 0 0 139 0.5
./dataset/image

./dataset/images/brown SandyBrown 244 164 96 0.8
./dataset/images/brown SandyBrown 244 164 96 0.9
./dataset/images/brown SandyBrown 244 164 96 1.0
./dataset/images/brown SandyBrown 244 164 96 1.1
./dataset/images/brown SandyBrown 244 164 96 1.2
./dataset/images/brown SandyBrown 244 164 96 1.3
./dataset/images/brown SandyBrown 244 164 96 1.4
./dataset/images/brown SandyBrown 244 164 96 1.5
./dataset/images/brown SandyBrown 244 164 96 1.6
./dataset/images/brown SandyBrown 244 164 96 1.7
./dataset/images/brown SandyBrown 244 164 96 1.8
./dataset/images/brown Goldenrod 218 165 32 0.5
./dataset/images/brown Goldenrod 218 165 32 0.6
./dataset/images/brown Goldenrod 218 165 32 0.7
./dataset/images/brown Goldenrod 218 165 32 0.8
./dataset/images/brown Goldenrod 218 165 32 0.9
./dataset/images/brown Goldenrod 218 165 32 1.0
./dataset/images/brown Goldenrod 218 165 32 1.1
./dataset/images/brown Goldenrod 218 165 32 1.2
./dataset/images/brown Goldenrod 218 165 32 1.3
./dataset/images/brown Golden

./dataset/images/white Azure 240 255 255 0.8
./dataset/images/white Azure 240 255 255 0.9
./dataset/images/white Azure 240 255 255 1.0
./dataset/images/white Azure 240 255 255 1.1
./dataset/images/white Azure 240 255 255 1.2
./dataset/images/white Azure 240 255 255 1.3
./dataset/images/white Azure 240 255 255 1.4
./dataset/images/white Azure 240 255 255 1.5
./dataset/images/white Azure 240 255 255 1.6
./dataset/images/white Azure 240 255 255 1.7
./dataset/images/white Azure 240 255 255 1.8
./dataset/images/white AliceBlue 240 248 255 0.5
./dataset/images/white AliceBlue 240 248 255 0.6
./dataset/images/white AliceBlue 240 248 255 0.7
./dataset/images/white AliceBlue 240 248 255 0.8
./dataset/images/white AliceBlue 240 248 255 0.9
./dataset/images/white AliceBlue 240 248 255 1.0
./dataset/images/white AliceBlue 240 248 255 1.1
./dataset/images/white AliceBlue 240 248 255 1.2
./dataset/images/white AliceBlue 240 248 255 1.3
./dataset/images/white AliceBlue 240 248 255 1.4
./dataset/image

./dataset/images/white MistyRose 255 228 225 1.0
./dataset/images/white MistyRose 255 228 225 1.1
./dataset/images/white MistyRose 255 228 225 1.2
./dataset/images/white MistyRose 255 228 225 1.3
./dataset/images/white MistyRose 255 228 225 1.4
./dataset/images/white MistyRose 255 228 225 1.5
./dataset/images/white MistyRose 255 228 225 1.6
./dataset/images/white MistyRose 255 228 225 1.7
./dataset/images/white MistyRose 255 228 225 1.8
./dataset/images/gray Gainsboro 220 220 220 0.5
./dataset/images/gray Gainsboro 220 220 220 0.6
./dataset/images/gray Gainsboro 220 220 220 0.7
./dataset/images/gray Gainsboro 220 220 220 0.8
./dataset/images/gray Gainsboro 220 220 220 0.9
./dataset/images/gray Gainsboro 220 220 220 1.0
./dataset/images/gray Gainsboro 220 220 220 1.1
./dataset/images/gray Gainsboro 220 220 220 1.2
./dataset/images/gray Gainsboro 220 220 220 1.3
./dataset/images/gray Gainsboro 220 220 220 1.4
./dataset/images/gray Gainsboro 220 220 220 1.5
./dataset/images/gray Gainsboro

In [15]:
#files_count(images_folder+'yellow')

max_images_count = 2000

for c in next(os.walk(images_folder))[1]:
    full_path = images_folder+c #images_folder+c
    img_count = images_count(full_path) # files_color_count = 
    print(c, img_count)
    
    for i in range(img_count, max_images_count):
        image_to_copy = os.listdir(full_path)[random.randint(0, img_count-1)]
        image_name_to_copy, image_extension = os.path.splitext(image_to_copy)
        
        img_original = Image.open(full_path+'/'+image_to_copy)
        width, height = img_original.size
        
        colors = img_original.getpixel((1, 1)) #((random.randint(0, width-1), random.randint(0, height-1)))
        r = colors[0]
        g = colors[1]
        b = colors[2]
        

        create_image(
            full_path, 
            image_name_to_copy+'_'+str(i), 
            r = r, 
            g = g, 
            b = b, 
            brightness=random.randint(5, 19)/10)        
        
        
        print(c, image_to_copy, image_name_to_copy, image_extension, r, g, b)
        
        if images_count(full_path) >= max_images_count:
            break

'''
    print(c, files_count(images_folder+'yellow')) # 
    

    files_count = files_count(images_folder+c)
    
    for i in range(files_count(images_folder + c), 1200):
        print(c, os.listdir(''.join([images_folder, c]))[1])
'''

blue 2001
brown 714
./dataset/images/brown Sienna_1_2_dither_714 160 82 45 0.5
brown Sienna_1_2_dither.png Sienna_1_2_dither .png 160 82 45
./dataset/images/brown SaddleBrown_1_0_dither_715 139 69 19 1.3
brown SaddleBrown_1_0_dither.png SaddleBrown_1_0_dither .png 139 69 19
./dataset/images/brown Bisque_0_5_low_716 255 228 196 1.1
brown Bisque_0_5_low.png Bisque_0_5_low .png 255 228 196
./dataset/images/brown DarkGoldenrod_1_8_dither_717 184 134 11 1.1
brown DarkGoldenrod_1_8_dither.png DarkGoldenrod_1_8_dither .png 184 134 11
./dataset/images/brown Brown_1_3_dither_718 165 42 42 0.6
brown Brown_1_3_dither.png Brown_1_3_dither .png 165 42 42
./dataset/images/brown SaddleBrown_1_8_719 139 69 19 1.5
brown SaddleBrown_1_8.png SaddleBrown_1_8 .png 139 69 19
./dataset/images/brown Sienna_0_6_dither_720 160 82 45 0.7
brown Sienna_0_6_dither.png Sienna_0_6_dither .png 160 82 45
./dataset/images/brown Maroon_1_4_721 128 0 0 0.6
brown Maroon_1_4.png Maroon_1_4 .png 128 0 0
./dataset/images/brow

brown NavajoWhite_1_2_low_733_1_8_low.png NavajoWhite_1_2_low_733_1_8_low .png 255 222 173
./dataset/images/brown Peru_1_7_dither_782 205 133 63 1.0
brown Peru_1_7_dither.png Peru_1_7_dither .png 205 133 63
./dataset/images/brown NavajoWhite_1_1_low_783 255 222 173 1.5
brown NavajoWhite_1_1_low.png NavajoWhite_1_1_low .png 255 222 173
./dataset/images/brown BurlyWood_1_6_dither_784 222 184 135 1.6
brown BurlyWood_1_6_dither.png BurlyWood_1_6_dither .png 222 184 135
./dataset/images/brown SaddleBrown_0_9_low_785 139 69 19 1.8
brown SaddleBrown_0_9_low.png SaddleBrown_0_9_low .png 139 69 19
./dataset/images/brown Peru_1_3_786 205 133 63 1.1
brown Peru_1_3.png Peru_1_3 .png 205 133 63
./dataset/images/brown Bisque_1_2_low_750_1_6_dither_787 255 228 196 1.8
brown Bisque_1_2_low_750_1_6_dither.png Bisque_1_2_low_750_1_6_dither .png 255 228 196
./dataset/images/brown SaddleBrown_0_9_low_741_1_2_788 139 69 19 1.7
brown SaddleBrown_0_9_low_741_1_2.png SaddleBrown_0_9_low_741_1_2 .png 139 69 19

./dataset/images/brown Bisque_1_2_low_750_1_6_dither_757_1_3_dither_791_0_7_840 255 228 196 1.4
brown Bisque_1_2_low_750_1_6_dither_757_1_3_dither_791_0_7.png Bisque_1_2_low_750_1_6_dither_757_1_3_dither_791_0_7 .png 255 228 196
./dataset/images/brown Bisque_1_2_low_750_1_6_841 255 228 196 1.8
brown Bisque_1_2_low_750_1_6.png Bisque_1_2_low_750_1_6 .png 255 228 196
./dataset/images/brown DarkGoldenrod_1_0_low_842 184 134 11 1.5
brown DarkGoldenrod_1_0_low.png DarkGoldenrod_1_0_low .png 184 134 11
./dataset/images/brown Bisque_1_2_low_750_1_6_dither_757_1_3_dither_791_0_7_dither_843 255 228 196 1.2
brown Bisque_1_2_low_750_1_6_dither_757_1_3_dither_791_0_7_dither.png Bisque_1_2_low_750_1_6_dither_757_1_3_dither_791_0_7_dither .png 255 228 196
./dataset/images/brown DarkGoldenrod_1_6_dither_844 184 134 11 0.6
brown DarkGoldenrod_1_6_dither.png DarkGoldenrod_1_6_dither .png 184 134 11
./dataset/images/brown NavajoWhite_1_2_low_733_1_8_799_0_8_813_1_9_dither_845 204 177 138 1.3
brown Navaj

brown Chocolate_1_1_818_1_0_dither_819_0_8_dither.png Chocolate_1_1_818_1_0_dither_819_0_8_dither .png 210 105 30
./dataset/images/brown Goldenrod_0_8_low_893 218 165 32 1.9
brown Goldenrod_0_8_low.png Goldenrod_0_8_low .png 218 165 32
./dataset/images/brown Chocolate_1_3_dither_894 210 105 30 1.2
brown Chocolate_1_3_dither.png Chocolate_1_3_dither .png 210 105 30
./dataset/images/brown Chocolate_0_6_low_726_0_5_low_763_1_4_777_0_6_low_895 210 105 30 1.5
brown Chocolate_0_6_low_726_0_5_low_763_1_4_777_0_6_low.png Chocolate_0_6_low_726_0_5_low_763_1_4_777_0_6_low .png 210 105 30
./dataset/images/brown Goldenrod_1_6_dither_817_1_8_low_896 218 165 32 1.4
brown Goldenrod_1_6_dither_817_1_8_low.png Goldenrod_1_6_dither_817_1_8_low .png 218 165 32
./dataset/images/brown Brown_1_3_dither_718_0_6_897 165 42 42 0.7
brown Brown_1_3_dither_718_0_6.png Brown_1_3_dither_718_0_6 .png 165 42 42
./dataset/images/brown Cornsilk_0_7_low_898 255 248 220 1.5
brown Cornsilk_0_7_low.png Cornsilk_0_7_low .pn

./dataset/images/brown Cornsilk_1_4_dither_875_1_8_947 255 248 220 0.9
brown Cornsilk_1_4_dither_875_1_8.png Cornsilk_1_4_dither_875_1_8 .png 255 248 220
./dataset/images/brown Cornsilk_1_7_dither_948 255 248 220 1.9
brown Cornsilk_1_7_dither.png Cornsilk_1_7_dither .png 255 248 220
./dataset/images/brown BlanchedAlmond_0_6_low_899_1_2_dither_906_0_8_low_949 255 235 205 0.9
brown BlanchedAlmond_0_6_low_899_1_2_dither_906_0_8_low.png BlanchedAlmond_0_6_low_899_1_2_dither_906_0_8_low .png 255 235 205
./dataset/images/brown Cornsilk_1_7_dither_950 255 248 220 0.5
brown Cornsilk_1_7_dither.png Cornsilk_1_7_dither .png 255 248 220
./dataset/images/brown Bisque_1_4_dither_830_1_1_dither_951 255 228 196 0.5
brown Bisque_1_4_dither_830_1_1_dither.png Bisque_1_4_dither_830_1_1_dither .png 255 228 196
./dataset/images/brown Bisque_1_5_dither_952 255 228 196 1.7
brown Bisque_1_5_dither.png Bisque_1_5_dither .png 255 228 196
./dataset/images/brown BurlyWood_0_6_dither_810_0_9_865_1_5_low_953 222 1

...

yellow DarkKhaki_0_8_dither_508_1_6_low_605_1_4_dither.png DarkKhaki_0_8_dither_508_1_6_low_605_1_4_dither .png 189 183 107
./dataset/images/yellow DarkKhaki_0_8_dither_816_1_2_low_838 189 183 107 1.9
yellow DarkKhaki_0_8_dither_816_1_2_low.png DarkKhaki_0_8_dither_816_1_2_low .png 189 183 107
./dataset/images/yellow DarkKhaki_0_8_dither_508_1_6_755_1_7_dither_786_1_0_dither_799_1_0_839 189 183 107 1.9
yellow DarkKhaki_0_8_dither_508_1_6_755_1_7_dither_786_1_0_dither_799_1_0.png DarkKhaki_0_8_dither_508_1_6_755_1_7_dither_786_1_0_dither_799_1_0 .png 189 183 107
./dataset/images/yellow DarkKhaki_1_1_low_507_1_0_low_537_1_7_dither_840 189 183 107 1.6
yellow DarkKhaki_1_1_low_507_1_0_low_537_1_7_dither.png DarkKhaki_1_1_low_507_1_0_low_537_1_7_dither .png 189 183 107
./dataset/images/yellow DarkKhaki_0_6_low_841 189 183 107 0.9
yellow DarkKhaki_0_6_low.png DarkKhaki_0_6_low .png 189 183 107
./dataset/images/yellow DarkKhaki_1_5_dither_523_1_5_565_0_5_dither_634_0_9_dither_842 189 183 107 

yellow DarkKhaki_1_0_dither_485_0_8_494_0_7.png DarkKhaki_1_0_dither_485_0_8_494_0_7 .png 189 183 107
./dataset/images/yellow DarkKhaki_0_8_dither_816_1_2_low_838_1_9_low_881 189 183 107 1.1
yellow DarkKhaki_0_8_dither_816_1_2_low_838_1_9_low.png DarkKhaki_0_8_dither_816_1_2_low_838_1_9_low .png 189 183 107
./dataset/images/yellow DarkKhaki_0_8_dither_508_1_6_low_605_1_4_753_0_9_dither_882 189 183 107 0.6
yellow DarkKhaki_0_8_dither_508_1_6_low_605_1_4_753_0_9_dither.png DarkKhaki_0_8_dither_508_1_6_low_605_1_4_753_0_9_dither .png 189 183 107
./dataset/images/yellow DarkKhaki_1_1_low_507_1_0_low_513_1_9_low_883 189 183 107 1.4
yellow DarkKhaki_1_1_low_507_1_0_low_513_1_9_low.png DarkKhaki_1_1_low_507_1_0_low_513_1_9_low .png 189 183 107
./dataset/images/yellow DarkKhaki_0_8_dither_884 189 183 107 1.1
yellow DarkKhaki_0_8_dither.png DarkKhaki_0_8_dither .png 189 183 107
./dataset/images/yellow DarkKhaki_0_8_dither_508_1_6_755_1_7_dither_786_1_0_866_1_9_low_885 189 183 107 1.3
yellow Dar

yellow DarkKhaki_0_8_dither_508_1_6_dither_756_0_6_low_768_1_3_low.png DarkKhaki_0_8_dither_508_1_6_dither_756_0_6_low_768_1_3_low .png 189 183 107
./dataset/images/yellow DarkKhaki_1_1_dither_541_0_8_low_921 255 255 0 1.7
yellow DarkKhaki_1_1_dither_541_0_8_low.png DarkKhaki_1_1_dither_541_0_8_low .png 255 255 0
./dataset/images/yellow DarkKhaki_0_8_low_820_0_5_low_922 189 183 107 1.7
yellow DarkKhaki_0_8_low_820_0_5_low.png DarkKhaki_0_8_low_820_0_5_low .png 189 183 107
./dataset/images/yellow DarkKhaki_1_1_dither_541_0_8_dither_737_1_1_824_1_8_low_923 255 255 0 1.7
yellow DarkKhaki_1_1_dither_541_0_8_dither_737_1_1_824_1_8_low.png DarkKhaki_1_1_dither_541_0_8_dither_737_1_1_824_1_8_low .png 255 255 0
./dataset/images/yellow DarkKhaki_0_5_dither_781_1_8_low_846_1_6_low_924 189 183 107 1.2
yellow DarkKhaki_0_5_dither_781_1_8_low_846_1_6_low.png DarkKhaki_0_5_dither_781_1_8_low_846_1_6_low .png 189 183 107
./dataset/images/yellow DarkKhaki_0_8_dither_508_1_6_755_1_7_925 189 183 107 0.8

yellow DarkKhaki_0_8_dither_508_1_6_755_1_7_low_941_0_6_dither.png DarkKhaki_0_8_dither_508_1_6_755_1_7_low_941_0_6_dither .png 189 183 107
./dataset/images/yellow DarkKhaki_1_0_dither_485_0_8_494_0_7_828_1_8_960 189 183 107 0.6
yellow DarkKhaki_1_0_dither_485_0_8_494_0_7_828_1_8.png DarkKhaki_1_0_dither_485_0_8_494_0_7_828_1_8 .png 189 183 107
./dataset/images/yellow DarkKhaki_0_5_low_739_1_1_859_0_7_dither_961 189 183 107 1.7
yellow DarkKhaki_0_5_low_739_1_1_859_0_7_dither.png DarkKhaki_0_5_low_739_1_1_859_0_7_dither .png 189 183 107
./dataset/images/yellow DarkKhaki_1_0_dither_485_0_8_494_0_7_880_1_5_958_0_7_dither_962 189 183 107 0.8
yellow DarkKhaki_1_0_dither_485_0_8_494_0_7_880_1_5_958_0_7_dither.png DarkKhaki_1_0_dither_485_0_8_494_0_7_880_1_5_958_0_7_dither .png 189 183 107
./dataset/images/yellow DarkKhaki_0_9_dither_603_1_2_782_0_5_dither_829_0_8_low_963 189 183 107 0.5
yellow DarkKhaki_0_9_dither_603_1_2_782_0_5_dither_829_0_8_low.png DarkKhaki_0_9_dither_603_1_2_782_0_5_di

"\n    print(c, files_count(images_folder+'yellow')) # \n    \n\n    files_count = files_count(images_folder+c)\n    \n    for i in range(files_count(images_folder + c), 1200):\n        print(c, os.listdir(''.join([images_folder, c]))[1])\n"